In [1]:
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'user_name', # имя пользователя (подставлено шаблонное значение в целях сохранения интеллектуальной собственности)
 'pwd': 'password', # пароль (аналогично)
 'host': 'example.mdb.yandexcloud.net',
 'port': 0000, # порт подключения (аналогично)
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

# SQL. Анализ баз данных.

__Цель:__

Проанализировать базу данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.


__Задачи:__

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

__Схема данных:__

Доступно по ссылке: https://drive.google.com/file/d/1IunZQ_2E21TzTjWeWS6liiYmCpu1PNTF/view?usp=sharing

__Описание данных:__

Таблица `books`:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors`:

- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers`:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings`:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews`:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

<br>

## Исследование таблиц

In [2]:
# вывод таблицы из базы Книги
query = '''
select
    *
from 
    books
limit
    5;
'''

books = pd.io.sql.read_sql(query, con = engine)
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 150
books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = '''
select
    count(*) as cnt
from
    books;
'''

df = pd.io.sql.read_sql(query, con = engine)
df

,cnt
0,1000


In [4]:
# вывод таблицы из базы Авторы
query = '''
select
    *
from 
    authors
limit
    5;
'''

authors = pd.io.sql.read_sql(query, con = engine) 
authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''
select
    count(*) as cnt
from
    authors;
'''

df = pd.io.sql.read_sql(query, con = engine)
df

,cnt
0,636


In [6]:
# вывод таблицы из базы Издательства
query = '''
select
    *
from 
    publishers
limit
    5;
'''

publishers = pd.io.sql.read_sql(query, con = engine) 
publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [7]:
query = '''
select
    count(*) as cnt
from
    publishers;
'''

df = pd.io.sql.read_sql(query, con = engine)
df

,cnt
0,340


In [8]:
# вывод таблицы из базы Рейтинги
query = '''
select
    *
from 
    ratings
limit
    5;
'''

ratings = pd.io.sql.read_sql(query, con = engine) 
ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
query = '''
select
    count(*) as cnt
from
    ratings;
'''

df = pd.io.sql.read_sql(query, con = engine)
df

,cnt
0,6456


In [10]:
# вывод таблицы из базы Обзоры
query = '''
select
    *
from 
    reviews
limit
    5;
'''

reviews = pd.io.sql.read_sql(query, con = engine) 
reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


In [11]:
query = '''
select
    count(*) as cnt
from
    reviews;
'''

df = pd.io.sql.read_sql(query, con = engine)
df

,cnt
0,2793


### Вывод по итогам знакомства с таблицами

Нам доступны для анализа 5 таблиц с данными:

1. books -- 1 000 книг,
2. authors -- 636 авторов,
3. publishers -- 340 издательств,
4. ratings -- 6 456 оценок,
5. reviews -- 2 793 отзыва.


## Анализ БД

### Книги, вышедшие после 1 января 2000 года

In [12]:
query = '''
select
    count(*) as cnt
from 
    books
where
    publication_date > '2000-01-01'
'''

df = pd.io.sql.read_sql(query, con = engine) 
df

,cnt
0,819


#### Вывод

После 1 января 2000 года (не включительно) было выпущено 819 книг.

<br>

### Количество обзоров и средняя оценка по каждой книге

In [13]:
query = '''
select
    books.book_id as book_id,
    books.author_id as author_id,
    books.title as title,
    count(text) as reviews_cnt,
    avg(subquery.avg_rating) as avg_rating
from
    books
    left join reviews on reviews.book_id = books.book_id
    left join (
        select
            book_id,
            avg(rating) as avg_rating
        from
            ratings
        group by
            book_id
) as subquery on subquery.book_id = books.book_id     
group by
    books.book_id
order by
    reviews_cnt desc;
'''

df = pd.io.sql.read_sql(query, con = engine) 
df 

,book_id,author_id,title,reviews_cnt,avg_rating
0,948,554,Twilight (Twilight #1),7,3.662500
1,695,395,The Curious Incident of the Dog in the Night-Time,6,4.081081
2,299,236,Harry Potter and the Chamber of Secrets (Harry Potter #2),6,4.287500
3,696,106,The Da Vinci Code (Robert Langdon #2),6,3.830508
4,734,278,The Glass Castle,6,4.206897
...,...,...,...,...,...
995,672,144,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
996,808,350,The Natural Way to Draw,0,3.000000
997,221,151,Essential Tales and Poems,0,4.000000
998,83,174,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


#### Вывод

Для каждой книги посчитали количество обзоров и среднюю оценку. Лидер по количеству обзоров -- Twilight (Twilight #1).

<br>

### Издательство, которое выпустило наибольшее число книг толще 50 страниц


In [14]:
query = '''
select
    publishers.publisher as publisher,
    books_cnt
from
    publishers
    right join (
        select
            count(books.book_id) as books_cnt,
            books.publisher_id as publisher_id
        from
            books
        where
            books.num_pages > 50
        group by
            publisher_id
) as subquery on subquery.publisher_id = publishers.publisher_id
order by
    books_cnt desc
limit 1;
'''

df = pd.io.sql.read_sql(query, con = engine) 
df

,publisher,books_cnt
0,Penguin Books,42


#### Вывод

Определили издательство, которое выпустило наибольшее число книг толще 50 страниц. Им оказалось издательство Penguin Books.

<br>

### Автор с самой высокой средней оценкой книг


In [15]:
query = '''
select
    books.author_id as author_id,
    authors.author as author,
    avg(subquery.rating_avg) as rating_avg
from 
    books
    inner join (
        select
            book_id,
            avg(rating) as rating_avg
        from 
            ratings
        group by 
            book_id
        having 
            count(*) >= 50
        ) as subquery  on subquery.book_id = books.book_id
    inner join authors on authors.author_id = books.author_id
group by 
    books.author_id,
    authors.author
order by 
    rating_avg desc
limit 1;
'''

df = pd.io.sql.read_sql(query, con = engine) 
df

,author_id,author,rating_avg
0,236,J.K. Rowling/Mary GrandPré,4.283844


#### Вывод

Определили автора с самой высокой средней оценкой книг, учитывали только книги с 50 и более оценками. Это оказалась Джоан Роулинг, средний рейтинг которой составил 4.28.

<br>

### Cреднее количество обзоров от пользователей


In [16]:
query = '''
select
    avg(fifty.reviews_cnt) as avg_reviews
from
    (select
        username,
        count(text) as reviews_cnt
    from
        reviews
    group by
        username) as fifty
where
    fifty.username in (
        select
            ratings.username as username
        from
            ratings
        group by
            username
        having
            count(ratings.rating) > 50);
    '''

df = pd.io.sql.read_sql(query, con = engine)
df

,avg_reviews
0,24.333333


#### Вывод

Определили среднее количество обзоров от пользователей, которые поставили больше 50 оценок -- 24 обзора на 1 пользователя.

### Вывод

Проанализировали базу данных о размещенных на сервисе книгах. Всего размещено 819 книг, выпущенных после 1 января 2000 года.

Лидером по количеству обзоров является книга Twilight (Twilight #1).

Больше всего книг от издательства Penguin Books.

Среди размещенных книг, автора с самой высокой средней оценкой книг -- Джоан Роулинг, средний рейтинг которой составил 4.28.

В среднем каждый активный пользователь (поставил более 50 оценок) оставил 24 обзора.